In [3]:
from pydub.playback import play

In [23]:

import os
from pydub import AudioSegment, silence, effects
import uuid
from unidecode import unidecode
import json
import numpy as np
import io
import scipy.io.wavfile
import array


def get_audio_metadata(audio):
    meta =  {
        "sample_rate": audio.frame_rate,
        "channels": audio.channels, 
        "bytes_per_sample":audio.sample_width, # 1 means 8 bit, 2 means 16 bit   
    }

    return meta


def save_sample_meta(audio_meta, segment, out_dir):
    
    # declare metadata and sample dirs
    sample_dir = out_dir + '/samples/'
    meta_dir = out_dir + '/metadata/'
    
    if not os.path.exists(sample_dir):
        os.makedirs(sample_dir)
    if not os.path.exists(meta_dir):
        os.makedirs(meta_dir)
    
    sample_name = str(uuid.uuid4())
    seg_path = os.path.join(sample_dir, sample_name+'.wav')
    meta_path = os.path.join(meta_dir, sample_name+'.json')
    
    # here we can insert sample specific information on the metadata
    segment_meta_dict = audio_meta.copy()
    
    # Save the segment
    segment.export(seg_path, format='wav')
    
    # Save the metadata
    with open(meta_path, 'w', encoding='utf8') as fp:
        json.dump(segment_meta_dict, fp)
        
        
def segment_audio(
        file_path, out_dir, segment_length_s=10, target_sr=32000, n_channels=1,
        cutoff='pad', overlap=0.0, normalize=False, denoise=False, desilence=False
    ):

    # Get file name without extension for caption
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    file_name = unidecode(file_name.lower())
    
    # load audio
    audio = AudioSegment.from_file(file_path)
    
    # resample
    if target_sr is not None:
        audio = audio.set_frame_rate(target_sr) # resample to target_sr
    
    # force mono/stereo          
    if n_channels is not None:
        audio = audio.set_channels(n_channels) # convert to mono
    
    if normalize:
        pass
    
    # remove noise
    
    # remove silence
    if desilence:
        audio = remove_silence(audio)
    
    # Audio info dict
    audio_metadata = get_audio_metadata(audio, file_name) 
    audio_metadata["title"] = file_name.split('.')[0]
    
    # segment into segment_length_s samples
    segment_length_ms = segment_length_s * 1000
    step = int((1-overlap)*segment_length_ms)
    for i in range(0, len(audio), step):
        # create segment
        start_time = i 
        end_time = i + segment_length_ms
        segment = audio[start_time:end_time]

        # pad or crop end-of-file samples
        if end_time > len(audio):
            if cutoff=='pad': # pad with silence (0 amplitude)
                if len(segment) < 0.5*segment_length_ms: # pad at most 50% of the signal
                    break
                pad_len = segment_length_ms - len(segment)
                silence = AudioSegment.silent(  
                    duration=pad_len,
                    frame_rate=segment.frame_rate
                )
                segment = segment + silence
            
            elif cutoff=='leave': # save sample smaller than segment_length_s
                segment = audio[start_time:len(audio)] 
            
            elif cutoff=='crop': # discard smaller sample
                break
            
        save_sample_meta(out_dir)
        
def remove_silence(audio_segment):
    silence_list = silence.detect_silence()
    
def normalize(audio_segment, kind='pydub'):
    if kind == 'pydub':
        return effects.normalize(audio_segment)
    
    elif kind == 'jukebox':
        np_segment = np.array(audio_segment.get_array_of_samples(), dtype=np.float16).reshape((-1, audio_segment.channels)).T
        
        if np_segment.ndim == 1:
            np_segment = np_segment[np.newaxis]
        np_segment = np_segment.mean(axis=0)
        
        # normalize audio
        norm_factor = np.abs(np_segment).max()
        if norm_factor > 0:
            np_segment /= norm_factor
        np_segment = np_segment.flatten()
        
        wav_io = io.BytesIO()
        scipy.io.wavfile.write(wav_io, audio_segment.frame_rate, np_segment)
        wav_io.seek(0)
        return AudioSegment.from_wav(wav_io)
        
        # array_segment = array.array(audio_segment.array_type, np_segment)
        # return audio_segment._spawn(array_segment)       
        

In [20]:
audio = AudioSegment.from_file("/home/cvillela/dataland/data/hanwha/training/ds/samples/0a00e51a-fb1a-42b3-b471-60c656bd3255.wav")

In [ ]:
play(audio)

In [ ]:
audio = normalize(audio, kind='pydub')
play(audio)

In [24]:
audio = normalize(audio, kind='jukebox')

/tmp/ipykernel_17538/4153650446.py:118: RuntimeWarning: overflow encountered in cast
  np_segment = np.array(audio_segment.get_array_of_samples(), dtype=np.float16).reshape((-1, audio_segment.channels)).T
/tmp/ipykernel_17538/4153650446.py:127: RuntimeWarning: invalid value encountered in divide
  np_segment /= norm_factor


TypeError: 'numpy.float16' object cannot be interpreted as an integer